In [ ]:
# ============================================
# SAR Image Download Script for Bajo Cauca - Confluencia
# ============================================

# --------- Install & Import Dependencies ---------
!pip install rasterio
from google.colab import drive
import ee
import geemap
import os

# --------- Mount Google Drive ---------
drive.mount('/content/drive')

# --------- Authenticate and Initialize Earth Engine ---------
ee.Authenticate()
ee.Initialize(project='YOUR-GEE-PROJECT-ID')  # <-- CHANGE THIS


In [ ]:
# --------- User Parameters ---------
# Define Area of Interest (AOI): Confluencia
ZONE = 'CONFLUENCIA'

# Rectangle coordinates: (East, South), (West, North)
LON_E, LAT_S = -74.746023, 8.037527  # Bottom-right
LON_W, LAT_N = -74.8303614, 8.113206  # Top-left

ROI = ee.Geometry.Rectangle([LON_W, LAT_S, LON_E, LAT_N])

# Year range
START_YEAR = 2017
END_YEAR = 2024

# Polarization & Orbit Direction
POLARIZATION = 'VV'
ORBIT = 'DESCENDING'

In [ ]:
# --------- function to get sar collection ---------

def get_sar_collection(start_date, end_date, roi, pol=POLARIZATION, orbit=ORBIT):
    sar_colection = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.eq('orbitProperties_pass', orbit))
            .filterBounds(roi)
            .filterDate(start_date, end_date)
            .select(pol))
    return sar_colection


In [ ]:
# --------- Create Folders ---------
BASE_DIR = f'/content/drive/MyDrive/{ZONE}' # <-- Edit path if needed
os.makedirs(f"{BASE_DIR}/PURA", exist_ok=True)
os.makedirs(f"{BASE_DIR}/VISUAL", exist_ok=True)

In [ ]:
# ---------- Download first image each month ----------
for year in range(START_YEAR, END_YEAR + 1):
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"
        end_date = f"{year}-{month:02d}-28"

        monthly_sar = get_sar_collection(start_date, end_date, ROI)
        first_image = monthly_sar.first()
        first_visual = first_image.visualize(**{'min' : -25, 'max' : 5})

        # Export pure image
        geemap.ee_export_image(
            first_image.clip(ROI),
            filename=f"{BASE_DIR}/PURA/{year}-{month:02d}.tif",
            scale=10, region=ROI, crs='EPSG:4326', file_per_band=False)

        # Export visualized image
        geemap.ee_export_image(
            first_visual.clip(ROI),
            filename=f"{BASE_DIR}/VV_VISUAL/{year}-{month:02d}.tif",
            scale=10, region=ROI, crs='EPSG:4326', file_per_band=False)

print("✅ Done: Images for each month downloaded.")